## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Dingle,PH,2020-08-30 18:03:53,11.00,122.67,77.56,84,100,5.10,overcast clouds
1,1,Qinhuangdao,CN,2020-08-30 18:03:53,39.93,119.59,69.67,84,13,1.25,few clouds
2,2,Tawau,MY,2020-08-30 18:03:53,4.25,117.89,77.00,88,20,3.36,few clouds
3,3,Arman,RU,2020-08-30 18:03:54,59.70,150.17,44.80,61,0,4.97,clear sky
4,4,Albany,US,2020-08-30 18:03:02,42.60,-73.97,71.01,43,22,4.00,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like on your trip? "))
max_temp = float(input("What is the maximum temperature you would like on your trip? "))

What is the minimum temperature you would like on your trip? 70
What is the maximum temperature you would like on your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Dingle,PH,2020-08-30 18:03:53,11.00,122.67,77.56,84,100,5.10,overcast clouds
2,2,Tawau,MY,2020-08-30 18:03:53,4.25,117.89,77.00,88,20,3.36,few clouds
4,4,Albany,US,2020-08-30 18:03:02,42.60,-73.97,71.01,43,22,4.00,few clouds
7,7,Ponta Do Sol,PT,2020-08-30 18:03:55,32.67,-17.10,75.00,42,5,5.17,clear sky
9,9,Kapaa,US,2020-08-30 17:59:49,22.08,-159.32,79.00,74,40,16.11,scattered clouds
13,13,Gorom-Gorom,BF,2020-08-30 18:03:57,14.44,-0.24,88.11,55,96,9.33,light rain
15,15,Vaini,TO,2020-08-30 18:03:58,-21.20,-175.20,75.20,88,20,9.17,few clouds
16,16,Atuona,PF,2020-08-30 18:03:58,-9.80,-139.03,78.58,79,48,22.68,light rain
17,17,Georgetown,MY,2020-08-30 18:03:58,5.41,100.34,80.60,94,40,3.36,scattered clouds
18,18,Lodhikheda,IN,2020-08-30 18:03:59,21.58,78.83,75.20,94,20,3.44,haze


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Dingle,PH,77.56,overcast clouds,11.00,122.67,
2,Tawau,MY,77.00,few clouds,4.25,117.89,
4,Albany,US,71.01,few clouds,42.60,-73.97,
7,Ponta Do Sol,PT,75.00,clear sky,32.67,-17.10,
9,Kapaa,US,79.00,scattered clouds,22.08,-159.32,
13,Gorom-Gorom,BF,88.11,light rain,14.44,-0.24,
15,Vaini,TO,75.20,few clouds,-21.20,-175.20,
16,Atuona,PF,78.58,light rain,-9.80,-139.03,
17,Georgetown,MY,80.60,scattered clouds,5.41,100.34,
18,Lodhikheda,IN,75.20,haze,21.58,78.83,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.tail(10)
  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
690,Kilindoni,TZ,76.55,clear sky,-7.91,39.67,Bustani B&B
691,Panzhihua,CN,72.81,moderate rain,26.59,101.71,Electric Power Hotel
693,Srivardhan,IN,79.38,light rain,18.03,73.02,Savli Resort/Hotel - Shrivardhan
696,Gashua,NG,85.93,scattered clouds,12.87,11.05,Forte Oil - Old Lorry Park
698,Pangai,TO,77.00,broken clouds,-19.80,-174.35,Ha'apai Beach Resort
700,Appleton,US,72.00,scattered clouds,44.26,-88.42,La Quinta Inn & Suites by Wyndham Appleton Col...
701,Nouadhibou,MR,80.60,sand,20.93,-17.03,El Medina
705,Sibolga,ID,76.32,overcast clouds,1.74,98.78,Hotel Syariah CN Darussalam
706,Maturin,VE,87.94,light rain,9.75,-63.18,Residencias Guaritos I
710,Mandera,KE,86.40,scattered clouds,3.94,41.87,Kornesh Hotel and Lodging


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))